In [6]:
import tweepy
import yaml

with open('config.yml', "r") as stream:
    try:
        cf = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


auth = tweepy.OAuth1UserHandler(
    cf['twitter_key'], 
    cf['twitter_secret'], 
    cf['twitter_access_key'], 
    cf['twitter_access_secret']
)

In [7]:
api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

What's my place in the world population and how long will I live?
L: https://t.co/SP4ZvFkbZZ
C: https://t.co/Ts7U6iUEmN
Philippine central bank says keeping close watch on risks to inflation outlook https://t.co/EMyBeVLCx1 https://t.co/l86xvxzRy2
If you will solve problems until you’re blocked, you’re valuable.

If you will solve problems until you’re blocked,… https://t.co/43HOx8bZ6H
RT @MarchMadnessMBB: WHAT A GAME! 🍀

Notre Dame SURVIVES in 2OT and takes down Rutgers 89-87! #MarchMadness https://t.co/812HQRwW1P
RT @MarchMadnessMBB: FIRST TIME FEELING FOR WRIGHT STATE! 👏

The Raiders pick up their first #MarchMadness victory in school history after…
RT @MarchMadnessMBB: NOTRE DAME WINS IT IN DOUBLE OT!

THE LUCK OF THE IRISH ON ST. PATRICK'S DAY 🍀

@NDmbb #MarchMadness https://t.co/BmeI…
RT @MarchMadnessMBB: .@NDmbb with the clutch bucket to give them the lead 🔥 #MarchMadness https://t.co/QlAdgW0FAc
RT @MarchMadnessMBB: THIS. SEQUENCE! 🔥

We’re heading to DOUBLE OT in Dayton. #MarchM